# Welcome to the Lab 🥼🧪

#### How to Download Data as a CSV?

In this notebook, you will learn how to use the Parcl Labs API to search for markets and download data as CSV files. This guide will help you:

1. **Search for Markets:** Understand how to use the search feature to find markets and retrieve their unique identifiers (`parcl_ids`).
2. **Retrieve Data:** Use these `parcl_ids` to pull various types of data about the markets, including housing metrics, investor metrics, portfolio metrics, and more.
3. **Download CSV Files:** Save the retrieved data as CSV files for further analysis and use in tools like Excel.

The Parcl Labs API includes over 70,000 markets, each identified by a unique `parcl_id`. The search feature allows you to retrieve these `parcl_ids`, which you can then use to pull detailed data about specific markets using other API methods.

**Reminders:**

- You can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

- To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this Google Colab [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.
- To run this notebook at scale and download data for multiple markets and endpoints, you will need to upgrade your Parcl Labs API account from free to starter to get additional credits. You can easily upgrade at any time by visiting your [Parcl Labs dashboard](https://dashboard.parcllabs.com/login), clicking "Upgrade Now" ($99, no commitment). This will unlock more credits immediately.

Run in Collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/introduction/download_data.ipynb)

**Table of Contents Of Endpoint Data Downloads:**
1. **Market Metrics:**
   - [Housing Event Counts](https://docs.parcllabs.com/reference/housing_event_counts_v1_market_metrics__parcl_id__housing_event_counts_get-1)
   - [Housing Event Prices](https://docs.parcllabs.com/reference/housing_event_prices_v1_market_metrics__parcl_id__housing_event_prices_get-1)
   - [Housing Stock](https://docs.parcllabs.com/reference/housing_stock_v1_market_metrics__parcl_id__housing_stock_get-1)
   - [All Cash Transactions](https://docs.parcllabs.com/reference/all_cash_v1_market_metrics__parcl_id__all_cash_get)
   - [Housing Event Property Attributes](https://docs.parcllabs.com/reference/housing_event_property_attributes_v1_market_metrics__parcl_id__housing_event_property_attributes_get)
2. **For Sale Market Metrics:**
   - [New Listings Rolling Counts](https://docs.parcllabs.com/reference/new_listings_rolling_counts_v1_for_sale_market_metrics__parcl_id__new_listings_rolling_counts_get-1)
   - [For Sale Inventory](https://docs.parcllabs.com/reference/for_sale_inventory_v1_for_sale_market_metrics__parcl_id__for_sale_inventory_get)
   - [For Sale Inventory Price Changes](https://docs.parcllabs.com/reference/for_sale_inventory_price_changes_v1_for_sale_market_metrics__parcl_id__for_sale_inventory_price_changes_get)
3. **Rental Market Metrics:**
   - [Gross Yield](https://docs.parcllabs.com/reference/gross_yield_v1_rental_market_metrics__parcl_id__gross_yield_get-1)
   - [Rental Units Concentration](https://docs.parcllabs.com/reference/rental_units_concentration_v1_rental_market_metrics__parcl_id__rental_units_concentration_get)
   - [New Listings for Rent Rolling Counts](https://docs.parcllabs.com/reference/new_listings_for_rent_rolling_counts_v1_rental_market_metrics__parcl_id__new_listings_for_rent_rolling_counts_get-1)
4. **Investor Metrics:**
   - [Housing Event Counts](https://docs.parcllabs.com/reference/housing_event_counts_v1_investor_metrics__parcl_id__housing_event_counts_get-1)
   - [Purchase to Sale Ratio](https://docs.parcllabs.com/reference/purchase_to_sale_ratio_v1_investor_metrics__parcl_id__purchase_to_sale_ratio_get-1)
   - [New Listings for Sale Rolling Counts](https://docs.parcllabs.com/reference/new_listings_for_sale_rolling_counts_v1_investor_metrics__parcl_id__new_listings_for_sale_rolling_counts_get-1)
   - [Housing Stock Ownership](https://docs.parcllabs.com/reference/housing_stock_ownership_v1_investor_metrics__parcl_id__housing_stock_ownership_get-1)
   - [Housing Event Prices](https://docs.parcllabs.com/reference/housing_event_prices_v1_investor_metrics__parcl_id__housing_event_prices_get)
5. **Portfolio Metrics:**
   - [Single Family Housing Event Counts](https://docs.parcllabs.com/reference/sf_housing_event_counts_v1_portfolio_metrics__parcl_id__sf_housing_event_counts_get)
   - [Single Family Housing Stock Ownership](https://docs.parcllabs.com/reference/sf_housing_stock_ownership_v1_portfolio_metrics__parcl_id__sf_housing_stock_ownership_get-1)
   - [New Listings for Sale Rolling Counts](https://docs.parcllabs.com/reference/sf_new_listings_for_sale_rolling_counts_v1_portfolio_metrics__parcl_id__sf_new_listings_for_sale_rolling_counts_get)
   - [New Listings for Rent Rolling Counts](https://docs.parcllabs.com/reference/sf_new_listings_for_rent_rolling_counts_v1_portfolio_metrics__parcl_id__sf_new_listings_for_rent_rolling_counts_get)

In [2]:
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Check if running in Google Colab
if "google.colab" in sys.modules:
    from google.colab import userdata
    # Install required packages
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'parcllabs', 'plotly', 'kaleido'])
    # Retrieve API key from user data
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    # Retrieve API key from environment variable
    api_key = os.getenv('PARCL_LABS_API_KEY')

# Ensure the API key is available
if not api_key:
    raise ValueError("API key for Parcl Labs is not set. Please set the 'PARCL_LABS_API_KEY' environment variable or provide it in Google Colab's userdata.")

In [3]:
# Import parcllabs and create a client
import parcllabs
from parcllabs import ParclLabsClient

# Print the version of parcllabs
print(f"Parcl Labs Version: {parcllabs.__version__}")

Parcl Labs Version: 1.0.1


In [4]:
# Initialize the Parcl Labs client with a limit for all calls
client = ParclLabsClient(api_key, limit=12)

In [5]:
# Let's get some sample markets to download data for; everything starts with a search.
# We'll retrieve the top 12 most populous metropolitan statistical areas (MSAs) in the country.
# There are many different ways to search for markets. The markets defined here will be used to set the fields for downloaded data.
# For more details, please refer to the Search Notebook: https://github.com/ParclLabs/parcllabs-examples/blob/main/python/introduction/search.ipynb

markets = client.search.markets.retrieve(
    location_type='CBSA',  # Used: CBSA stands for Core-Based Statistical Area, which includes metropolitan areas. Example: 'CITY'
    sort_by='TOTAL_POPULATION',  # Used: We're sorting the results by total population to get the largest metros. Example: 'MEDIAN_INCOME'
    sort_order='DESC'  # Used: DESC means descending order, so the largest populations come first. Example: 'ASC'
    # Other available parameters:
    # region='EAST_NORTH_CENTRAL',  # Specify the region if needed.
    # state_abbreviation='CA',  # Filter by state abbreviation.
    # query='San Francisco',  # Search by market name or other keywords.
)

# Let's store the unique identifiers (parcl_ids) of these markets for later use.
# We'll need these IDs to download specific data for each market in subsequent steps.
parcl_ids = markets['parcl_id'].tolist()

# Display the top 5 markets to verify our search results.
markets.head(5)

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,2900187,USA,35620,None,"New York-Newark-Jersey City, Ny-Nj-Pa",None,None,CBSA,19908595,93610,0,1,1,1
1,2900078,USA,31080,None,"Los Angeles-Long Beach-Anaheim, Ca",None,None,CBSA,13111917,89105,0,1,1,1
2,2899845,USA,16980,None,"Chicago-Naperville-Elgin, Il-In-Wi",None,None,CBSA,9566955,85087,0,1,1,1
3,2899734,USA,19100,None,"Dallas-Fort Worth-Arlington, Tx",None,None,CBSA,7673379,83398,0,1,0,1
4,2899967,USA,26420,None,"Houston-The Woodlands-Sugar Land, Tx",None,None,CBSA,7142603,78061,0,1,0,0


In [6]:
# Set the start date for the entire notebook.
# This date defines the beginning of the data range we want to retrieve.
# You can change this date to alter the range of data that is returned.
START_DATE = '2023-01-01'

# Placeholder for the end date parameter.
# END_DATE = 'YYYY-MM-DD'  # Uncomment and set the end date if needed.

# For this example, we're only setting the start date to get data from the past year.

In [7]:
# Download data from the Housing Event Counts endpoint
results_housing_event_counts = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_counts = results_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.1s (10.52/s) 


,date,sales,new_listings_for_sale,new_rental_listings,parcl_id,property_type,name
0,2024-05-01,18573,14754,24797,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,20806,14475,21579,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,20148,13290,21845,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,18694,12080,20652,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,20935,11417,22525,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [8]:
# Save the data to the housing_event_counts CSV file locally
results_housing_event_counts.to_csv('housing_event_counts.csv', index=False)

In [9]:
# Download data from the Housing Stock endpoint
results_housing_stock = client.market_metrics.housing_stock.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_housing_stock = results_housing_stock.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_stock.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.74/s) 


,date,single_family,condo,townhouse,other,all_properties,parcl_id,name
0,2024-05-01,2803666,979745,78339,1586465,5448215,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,2803634,979282,78309,1586251,5447476,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,2803616,978899,78271,1586086,5446872,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,2803598,978474,78219,1585898,5446189,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,2803578,977940,78187,1585722,5445427,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the housing_stock CSV file locally
results_housing_stock.to_csv('housing_stock.csv', index=False)

In [12]:
# Download data from the Housing Event Prices endpoint
results_housing_event_prices = client.market_metrics.housing_event_prices.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_prices = results_housing_event_prices.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_prices.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.07/s) 


,date,price_median_sales,price_median_new_listings_for_sale,price_median_new_rental_listings,price_standard_deviation_sales,price_standard_deviation_new_listings_for_sale,price_standard_deviation_new_rental_listings,price_percentile_20th_sales,price_percentile_20th_new_listings_for_sale,price_percentile_20th_new_rental_listings,...,price_per_square_foot_standard_deviation_new_rental_listings,price_per_square_foot_percentile_20th_sales,price_per_square_foot_percentile_20th_new_listings_for_sale,price_per_square_foot_percentile_20th_new_rental_listings,price_per_square_foot_percentile_80th_sales,price_per_square_foot_percentile_80th_new_listings_for_sale,price_per_square_foot_percentile_80th_new_rental_listings,parcl_id,property_type,name
0,2024-05-01,650000,779000,3440,170830,272988,801,500000,590000,2750,...,2.12,287.30,299.50,2.69,507.81,586.54,6.38,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,625000,750000,3495,159971,267822,826,485000,579900,2770,...,2.21,278.01,297.62,2.73,499.29,587.21,6.74,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,610000,749990,3500,152481,269189,801,478000,560000,2800,...,2.17,270.27,293.10,2.80,486.54,589.37,6.77,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,600000,749995,3400,155414,284371,781,465000,550000,2724,...,2.34,267.00,295.12,2.68,484.92,613.20,6.32,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,610000,725000,3495,157578,249107,785,477000,549000,2800,...,2.16,270.88,294.12,2.70,484.70,599.81,6.45,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the housing_event_prices CSV file locally
results_housing_event_prices.to_csv('housing_event_prices.csv', index=False)

In [14]:
# Download data from the Housing Event Property Attributes endpoint
results_housing_event_property_attributes = client.market_metrics.housing_event_property_attributes.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_property_attributes = results_housing_event_property_attributes.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_property_attributes.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.41/s) 


,date,square_footage_median_sales,square_footage_median_new_listings_for_sale,square_footage_median_new_rental_listings,square_footage_standard_deviation_sales,square_footage_standard_deviation_new_listings_for_sale,square_footage_standard_deviation_new_rental_listings,square_footage_percentile_20th_sales,square_footage_percentile_20th_new_listings_for_sale,square_footage_percentile_20th_new_rental_listings,...,property_age_standard_deviation_new_rental_listings,property_age_percentile_20th_sales,property_age_percentile_20th_new_listings_for_sale,property_age_percentile_20th_new_rental_listings,property_age_percentile_80th_sales,property_age_percentile_80th_new_listings_for_sale,property_age_percentile_80th_new_rental_listings,parcl_id,property_type,name
0,2024-05-01,1608,1650,1300,353.15,359.58,333.29,1296,1296,1141,...,22.28,49,49,43,93,90,94,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,1610,1640,1296,349.54,363.76,319.26,1288,1284,1130,...,22.41,52,49,44,92,92,94,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,1616,1625,1300,346.90,366.10,324.24,1296,1281,1135,...,21.88,51,49,45,88,94,94,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,1606,1607,1300,353.99,361.40,323.11,1282,1275,1127,...,21.56,52,50,46,89,94,94,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,1615,1600,1300,350.16,361.82,315.47,1296,1260,1124,...,21.12,53,51,46,93,94,94,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the housing_event_property_attributes CSV file locally
results_housing_event_property_attributes.to_csv('housing_event_property_attributes.csv', index=False)

In [15]:
# Download data from the All Cash endpoint
results_all_cash = client.market_metrics.all_cash.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_all_cash = results_all_cash.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_all_cash.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.82/s) 


,date,count,pct_all_cash,parcl_id,property_type,name
0,2024-04-01,6188,29.74,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-03-01,6140,30.47,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-02-01,5606,29.99,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-01-01,6481,30.96,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2023-12-01,5742,29.03,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the all_cash CSV file locally
results_all_cash.to_csv('all_cash.csv', index=False)

In [16]:
# Download data from the Rental Market Metrics Gross Yield endpoint
results_gross_yield = client.rental_market_metrics.gross_yield.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_gross_yield = results_gross_yield.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_gross_yield.head()

|████████████████████████████████████████| 12/12 [100%] in 1.4s (8.81/s) 


,date,pct_gross_yield,parcl_id,property_type,name
0,2024-05-01,5.30,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,5.59,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,5.60,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,5.44,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,5.78,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the gross_yield CSV file locally
results_gross_yield.to_csv('gross_yield.csv', index=False)

In [17]:
# Download data from the Rental Market Metrics Rental Units Concentration endpoint
results_rental_units_concentration = client.rental_market_metrics.rental_units_concentration.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_rental_units_concentration = results_rental_units_concentration.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_rental_units_concentration.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.71/s) 


,date,rental_units,total_units,pct_rental_concentration,parcl_id,property_type,name
0,2024-05-01,738323,5448182,13.55,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,738231,5447443,13.55,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,738136,5446839,13.55,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,738012,5446156,13.55,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,737842,5445394,13.55,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the rental_units_concentration CSV file locally
results_rental_units_concentration.to_csv('rental_units_concentration.csv', index=False)

In [18]:
# Download data from the Rental Market Metrics New Listings for Rent Rolling Counts endpoint
results_new_listings_for_rent_rolling_counts = client.rental_market_metrics.new_listings_for_rent_rolling_counts.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_rent_rolling_counts = results_new_listings_for_rent_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_rent_rolling_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.59/s) 


,date,rolling_7_day,rolling_30_day,rolling_60_day,rolling_90_day,parcl_id,property_type,name
0,2024-06-10,6962,25809,51266,77738,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,5132,24732,50118,76567,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,6098,25736,50933,77283,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,6015,24566,51086,76441,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,5715,24088,50538,75867,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the new_listings_for_rent_rolling_counts CSV file locally
results_new_listings_for_rent_rolling_counts.to_csv('new_listings_for_rent_rolling_counts.csv', index=False)

In [20]:
# Download data from the For Sale Market Metrics New Listings Rolling Counts endpoint
results_new_listings_rolling_counts = client.for_sale_market_metrics.new_listings_rolling_counts.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_rolling_counts = results_new_listings_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_rolling_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.38/s) 


,date,rolling_7_day,rolling_30_day,rolling_60_day,rolling_90_day,parcl_id,property_type,name
0,2024-06-10,5380,17049,34610,51650,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,3592,15157,33074,49868,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,2691,15268,32796,49684,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,4098,16239,33355,50076,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,3457,15922,33141,49006,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the new_listings_rolling_counts CSV file locally
results_new_listings_rolling_counts.to_csv('new_listings_rolling_counts.csv', index=False)

In [22]:
# Download data from the For Sale Market Metrics For Sale Inventory Price Changes endpoint
results_for_sale_inventory_price_changes = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
)

# Include the parcl_id and market name in the DataFrame
results_for_sale_inventory_price_changes = results_for_sale_inventory_price_changes.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_for_sale_inventory_price_changes.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.87/s) 


,date,count_price_change,count_price_drop,median_days_bt_change,median_price_change,median_pct_price_change,pct_inventory_price_change,parcl_id,property_type,name
0,2024-06-10,13014,6842,30,-21200,-3.18,44.63,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,12348,6520,29,-24000,-3.23,44.93,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,12442,6440,29,-24000,-3.23,45.15,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,12644,6532,31,-24000,-3.28,44.54,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,12058,6221,32,-24000,-3.27,44.28,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the for_sale_inventory_price_changes CSV file locally
results_for_sale_inventory_price_changes.to_csv('for_sale_inventory_price_changes.csv', index=False)

In [23]:
# Download data from the Investor Metrics Housing Event Counts endpoint
results_investor_housing_event_counts = client.investor_metrics.housing_event_counts.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_investor_housing_event_counts = results_investor_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_investor_housing_event_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.70/s) 


,date,acquisitions,dispositions,new_listings_for_sale,new_rental_listings,parcl_id,name
0,2024-05-01,2531,2020,1439,1074,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,3004,2363,1347,779,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,3372,2218,1308,824,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,3228,2077,1233,727,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,3749,2339,1203,951,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the investor_housing_event_counts CSV file locally
results_investor_housing_event_counts.to_csv('investor_housing_event_counts.csv', index=False)

In [24]:
# Download data from the Investor Metrics Purchase to Sale Ratio endpoint
results_purchase_to_sale_ratio = client.investor_metrics.purchase_to_sale_ratio.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_purchase_to_sale_ratio = results_purchase_to_sale_ratio.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_purchase_to_sale_ratio.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (10.13/s) 


,date,purchase_to_sale_ratio,parcl_id,name
0,2024-05-01,1.25,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,1.27,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,1.52,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,1.55,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,1.60,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the purchase_to_sale_ratio CSV file locally
results_purchase_to_sale_ratio.to_csv('purchase_to_sale_ratio.csv', index=False)

In [25]:
# Download data from the Investor Metrics New Listings for Sale Rolling Counts endpoint
results_new_listings_for_sale_rolling_counts = client.investor_metrics.new_listings_for_sale_rolling_counts.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_sale_rolling_counts = results_new_listings_for_sale_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_sale_rolling_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.13/s) 


,date,count_rolling_7_day,count_rolling_30_day,count_rolling_60_day,count_rolling_90_day,pct_for_sale_market_rolling_7_day,pct_for_sale_market_rolling_30_day,pct_for_sale_market_rolling_60_day,pct_for_sale_market_rolling_90_day,parcl_id,property_type,name
0,2024-06-10,593,1737,3424,5092,11.02,10.19,9.89,9.86,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,334,1508,3203,4845,9.30,9.95,9.68,9.72,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,310,1506,3140,4848,11.52,9.86,9.57,9.76,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,381,1534,3140,4861,9.30,9.45,9.41,9.71,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,353,1530,3174,4811,10.21,9.61,9.58,9.82,2900187,ALL_PROPERTIES,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the new_listings_for_sale_rolling_counts CSV file locally
results_new_listings_for_sale_rolling_counts.to_csv('new_listings_for_sale_rolling_counts.csv', index=False)

In [26]:
# Download data from the Investor Metrics Housing Stock Ownership endpoint
results_housing_stock_ownership = client.investor_metrics.housing_stock_ownership.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_housing_stock_ownership = results_housing_stock_ownership.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_stock_ownership.head()

|████████████████████████████████████████| 12/12 [100%] in 1.1s (10.63/s) 


,date,count,pct_ownership,parcl_id,name
0,2024-05-01,352299,6.47,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,351312,6.45,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,350113,6.43,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,348896,6.41,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,347824,6.39,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the housing_stock_ownership CSV file locally
results_housing_stock_ownership.to_csv('housing_stock_ownership.csv', index=False)

In [27]:
# Download data from the Investor Metrics Housing Event Prices endpoint
results_investor_housing_event_prices = client.investor_metrics.housing_event_prices.retrieve(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
)

# Include the parcl_id and market name in the DataFrame
results_investor_housing_event_prices = results_investor_housing_event_prices.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_investor_housing_event_prices.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.11/s) 


,date,price_median_acquisitions,price_median_dispositions,price_median_new_listings_for_sale,price_median_new_rental_listings,price_per_square_foot_median_acquisitions,price_per_square_foot_median_dispositions,price_per_square_foot_median_new_listings_for_sale,price_per_square_foot_median_new_rental_listings,parcl_id,name
0,2024-05-01,565000,750000,875000,4100,338.54,420.52,417.45,3.60,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,515000,680000,945000,3850,323.97,387.96,442.48,3.36,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,560000,684000,889000,3600,322.02,382.95,420.48,3.05,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-02-01,553095,672000,889500,3600,314.94,374.96,428.82,2.91,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-01-01,596500,664833,849450,3850,326.92,388.44,400.82,3.15,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the investor_housing_event_prices CSV file locally
results_investor_housing_event_prices.to_csv('investor_housing_event_prices.csv', index=False)

In [28]:
# Download data from the Portfolio Metrics Single Family Housing Event Counts endpoint
results_sf_housing_event_counts = client.portfolio_metrics.sf_housing_event_counts.retrieve(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
)

# Include the parcl_id and market name in the DataFrame
results_sf_housing_event_counts = results_sf_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_sf_housing_event_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (10.02/s) 


,date,acquisitions,dispositions,new_listings_for_sale,new_rental_listings,parcl_id,portfolio_size,name
0,2024-05-01,1712,707,787,710,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,1865,1034,753,510,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,2109,763,712,495,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-01,2651,1582,1461,2087,2900078,ALL_PORTFOLIOS,"Los Angeles-Long Beach-Anaheim, Ca"
4,2024-04-01,2675,1761,1423,1954,2900078,ALL_PORTFOLIOS,"Los Angeles-Long Beach-Anaheim, Ca"


In [ ]:
# Save the data to the sf_housing_event_counts CSV file locally
results_sf_housing_event_counts.to_csv('sf_housing_event_counts.csv', index=False)

In [29]:
# Download data from the Portfolio Metrics Single Family Housing Stock Ownership endpoint
results_sf_housing_stock_ownership = client.portfolio_metrics.sf_housing_stock_ownership.retrieve(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
)

# Include the parcl_id and market name in the DataFrame
results_sf_housing_stock_ownership = results_sf_housing_stock_ownership.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_sf_housing_stock_ownership.head()

|████████████████████████████████████████| 12/12 [100%] in 1.2s (9.93/s) 


,date,count_portfolio_2_to_9,count_portfolio_10_to_99,count_portfolio_100_to_999,count_portfolio_1000_plus,count_all_portfolios,pct_sf_housing_stock_portfolio_2_to_9,pct_sf_housing_stock_portfolio_10_to_99,pct_sf_housing_stock_portfolio_100_to_999,pct_sf_housing_stock_portfolio_1000_plus,pct_sf_housing_stock_all_portfolios,parcl_id,name
0,2024-05-01,141860,8003,982,2105,152950,5.06,0.29,0.04,0.08,5.46,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-04-01,141252,7891,1078,2152,152373,5.04,0.28,0.04,0.08,5.44,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-03-01,135863,7295,1033,1998,146189,4.85,0.26,0.04,0.07,5.22,2900187,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-01,195982,10794,878,4350,212004,9.80,0.54,0.04,0.22,10.60,2900078,"Los Angeles-Long Beach-Anaheim, Ca"
4,2024-04-01,195506,10711,868,4348,211433,9.78,0.54,0.04,0.22,10.58,2900078,"Los Angeles-Long Beach-Anaheim, Ca"


In [ ]:
# Save the data to the sf_housing_stock_ownership CSV file locally
results_sf_housing_stock_ownership.to_csv('sf_housing_stock_ownership.csv', index=False)

In [31]:
# Download data from the Portfolio Metrics New Listings for Sale Rolling Counts endpoint
results_new_listings_for_sale_rolling_counts = client.portfolio_metrics.sf_new_listings_for_sale_rolling_counts.retrieve(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_sale_rolling_counts = results_new_listings_for_sale_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_sale_rolling_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.57/s) 


,date,count_rolling_7_day,count_rolling_30_day,count_rolling_60_day,count_rolling_90_day,pct_sf_for_sale_market_rolling_7_day,pct_sf_for_sale_market_rolling_30_day,pct_sf_for_sale_market_rolling_60_day,pct_sf_for_sale_market_rolling_90_day,parcl_id,portfolio_size,name
0,2024-06-10,254,802,1653,2440,12.09,10.63,10.53,10.67,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,171,733,1618,2360,10.09,10.35,10.58,10.57,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,124,723,1573,2350,9.90,10.20,10.56,10.65,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,195,783,1593,2406,10.52,10.43,10.59,10.90,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,187,772,1599,2365,11.35,10.38,10.77,11.02,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the portfolio_metrics_new_listings_for_sale_rolling_counts CSV file locally
results_new_listings_for_sale_rolling_counts.to_csv('portfolio_metrics_new_listings_for_sale_rolling_counts.csv', index=False)

In [32]:
# Download data from the Portfolio Metrics New Listings for Rent Rolling Counts endpoint
results_new_listings_for_rent_rolling_counts = client.portfolio_metrics.sf_new_listings_for_rent_rolling_counts.retrieve(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_rent_rolling_counts = results_new_listings_for_rent_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_rent_rolling_counts.head()

|████████████████████████████████████████| 12/12 [100%] in 1.3s (9.53/s) 


,date,count_rolling_7_day,count_rolling_30_day,count_rolling_60_day,count_rolling_90_day,pct_sf_for_rent_market_rolling_7_day,pct_sf_for_rent_market_rolling_30_day,pct_sf_for_rent_market_rolling_60_day,pct_sf_for_rent_market_rolling_90_day,parcl_id,portfolio_size,name
0,2024-06-10,62,456,814,1169,12.73,15.48,15.48,15.73,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-06-03,72,472,841,1190,17.39,15.52,16.00,15.82,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
2,2024-05-27,223,496,855,1224,17.06,15.62,16.02,15.91,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
3,2024-05-20,83,352,705,1093,13.63,15.30,15.57,15.77,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"
4,2024-05-13,78,348,702,1091,13.81,15.80,15.99,16.00,2900187,ALL_PORTFOLIOS,"New York-Newark-Jersey City, Ny-Nj-Pa"


In [ ]:
# Save the data to the portfolio_metrics_new_listings_for_rent_rolling_counts CSV file locally
results_new_listings_for_rent_rolling_counts.to_csv('portfolio_metrics_new_listings_for_rent_rolling_counts.csv', index=False)

In [ ]:
# For downloading Price Feed data, please refer to the Price Feed Notebook:
# https://github.com/ParclLabs/parcllabs-examples/blob/main/python/introduction/price_feed.ipynb